<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=e4c562ee695129975bb5ca7bdafbc0fa2f13cd63418c6e1aca14d61d1141893f
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-29 13:44:10
-------------------
qualified stocks: 91
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.28 L
Current:  1.37 C
-------------------
Today PnL: 52.40 K (0.38%)
Current PnL: -20.35 L (-13.69%)
CY Booked + Current PnL: -11.31 L (-7.61%)
-------------------
Total profit:  2.71 L
Total loss:  -23.07 L
-------------------
Total Booked + Current PnL: 15.67 L (12.88%)
Total Booked PnL: 36.02 L (29.6%)
Curr Year Booked PnL: 9.04 L (6.6%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 81.73 L (59.71%)
Deployed:  1.22 C
Current:  1.37 C
CAGR/XIRR %: 7.48%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HINDALCO,756.01,-18.25,56.0,H-LC,7.63,112640.0,8328.0,8324.0,0.37,7.98,7.39,15.96,9.0,1.00,0.85,26.23,X5K,ATH,METALS
53,MEDANTA,1486.00,13.45,61.0,H-MC,15.07,161936.0,35736.0,10445.0,-0.28,28.32,6.45,36.59,108.0,3.42,1.22,44.43,XY24,NTT,HEALTHCARE
81,TTKPRESTIG,770.00,100.21,61.0,M-SC,8.16,86224.0,-14553.0,14649.0,1.70,-14.44,16.99,0.09,245.0,-0.99,0.65,13.28,OX40N,NTT,DURABLES
4,APOLLOHOSP,8285.00,-16.30,52.0,H-LC,8.36,167343.0,11166.0,14927.0,-1.57,7.15,8.92,16.71,22.0,0.75,1.26,25.83,X40N,BTT,HEALTHCARE
66,SBIN,863.00,-14.20,41.0,M-LC,4.25,212742.0,11262.0,15956.0,0.11,5.59,7.50,13.51,60.0,0.71,1.60,18.89,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RAJOOENG,143.33,-32.99,38.0,H-SC,10.00,101530.0,-970.0,41800.0,0.14,-0.95,41.17,39.83,114.0,-0.02,0.77,0.52,AR,ATH,MISC
39,ICICIPRULI,790.00,-20.77,37.0,H-MC,2.38,136391.0,603.0,42145.0,-0.99,0.44,30.90,31.48,107.0,0.01,1.03,12.62,X40,ATH,INSURANCE
38,ICICIGI,2260.25,-20.58,36.0,H-MC,4.91,136745.0,612.0,30508.0,-0.67,0.45,22.31,22.86,91.0,0.02,1.03,13.12,X40,ATH,INSURANCE


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,7934.00,-42.20,35.0,H-LC,13.46,241752.0,-19867.0,139080.0,0.72,-7.59,57.53,45.57,7.0,-0.14,1.82,2.87,AR,NTT,ELECTRICAL
33,HCLTECH,1943.91,0.22,40.0,H-LC,10.19,229173.0,-12751.0,76017.0,0.68,-5.27,33.17,26.15,8.0,-0.17,1.73,8.30,X40,ATH,IT
47,JIOFIN,387.00,-0.91,42.0,H-LC,15.25,219240.0,1834.0,51653.0,1.00,0.84,23.56,24.61,48.0,0.04,1.65,56.12,XY24,BTT,FINANCE
87,VBL,671.64,-13.42,42.0,H-LC,8.07,311950.0,-3892.0,116545.0,-1.76,-1.23,37.36,35.67,5.0,-0.03,2.35,12.45,X40N,ATH,FMCG
84,UNITDSPR,1644.00,-14.09,45.0,H-LC,8.70,172894.0,-8865.0,44123.0,2.21,-4.88,25.52,19.39,86.0,-0.20,1.30,2.87,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,NESTLEIND,1377.0,-12.04,51.0,H-LC,2.82,270621.0,5195.0,51607.0,-0.51,1.96,19.07,21.40,11.0,0.10,2.04,9.43,XY25,NTT,FMCG
51,LTIM,7230.2,-6.94,48.0,H-LC,2.92,231075.0,-15076.0,94279.0,0.23,-6.12,40.80,32.18,16.0,-0.16,1.74,27.71,X200,ATH,IT
45,ITC,452.0,-37.92,50.0,H-LC,2.99,199977.0,-161.0,21058.0,2.01,-0.08,10.53,10.44,4.0,-0.01,1.51,5.48,X40,NTT,FMCG
20,CIPLA,1795.0,-18.41,63.0,H-LC,4.42,217857.0,13357.0,28060.0,0.63,6.53,12.88,20.25,10.0,0.48,1.64,15.84,X40N,BTT,PHARMA
5,ASIANPAINT,4250.0,-12.41,56.0,H-LC,7.22,220977.0,-30791.0,153027.0,0.89,-12.23,69.25,48.55,27.0,-0.20,1.67,18.86,X40,BTT,PAINTS


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-39.64,71.0,L-SC,26.76,89472.0,-24077.0,64125.0,9.54,-21.20,71.67,35.27,268.0,-0.38,0.67,118.98,XR,NTT,HOTELS
60,RELAXO,1176.00,-33.78,58.0,H-SC,8.51,61427.0,-54111.0,89100.0,7.19,-46.83,145.05,30.28,136.0,-0.61,0.46,20.77,X40N,NTT,FOOTWEAR
64,SAMMAANCAP,326.00,-165.91,55.0,M-SC,8.19,75438.0,-26772.0,120165.0,6.58,-26.19,159.29,91.37,208.0,-0.22,0.57,22.46,XY25,NTT,FINANCE
23,COLPAL,3767.14,0.29,58.0,H-MC,10.60,182528.0,-29213.0,111306.0,3.42,-13.80,60.98,38.77,84.0,-0.26,1.38,8.64,XY25,ATH,FMCG
18,CAMS,5211.76,-7.60,42.0,H-SC,6.23,104972.0,2968.0,40960.0,2.50,2.91,39.02,43.06,122.0,0.07,0.79,20.52,X40N,ATH,MISC


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VBL,671.64,-13.42,42.0,H-LC,8.07,311950.0,-3892.0,116545.0,-1.76,-1.23,37.36,35.67,5.0,-0.03,2.35,12.45,X40N,ATH,FMCG
76,TATAELXSI,9161.00,-26.23,19.0,H-MC,6.41,73500.0,-29152.0,54758.0,-1.65,-28.40,74.50,24.94,98.0,-0.53,0.55,11.73,OX40N,NTT,IT
4,APOLLOHOSP,8285.00,-16.30,52.0,H-LC,8.36,167343.0,11166.0,14927.0,-1.57,7.15,8.92,16.71,22.0,0.75,1.26,25.83,X40N,BTT,HEALTHCARE
85,VAIBHAVGBL,521.00,34.99,35.0,H-SC,12.10,90237.0,-61754.0,135879.0,-1.57,-40.63,150.58,48.77,125.0,-0.45,0.68,5.03,XR,NTT,JEWELLERY
43,INFY,2275.00,-19.24,45.0,H-LC,11.73,314644.0,1756.0,169939.0,-1.53,0.56,54.01,54.87,3.0,0.01,2.37,7.25,X40,BTT,IT


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TTKPRESTIG,770.0,100.21,61.0,M-SC,8.16,86224.0,-14553.0,14649.0,1.70,-14.44,16.99,0.09,245.0,-0.99,0.65,13.28,OX40N,NTT,DURABLES
82,UJJIVANSFB,60.0,82.03,41.0,H-SC,14.60,115182.0,-27297.0,46821.0,0.54,-19.16,40.65,13.70,163.0,-0.58,0.87,35.95,OX40N,NTT,BANKS
48,KANSAINER,340.0,-68.70,50.0,H-SC,3.46,216162.0,-53505.0,89837.0,0.90,-19.84,41.56,13.47,138.0,-0.60,1.63,9.33,XY24,NTT,PAINTS
19,CERA,9475.0,-20.53,41.0,H-SC,3.03,113337.0,-30466.0,57213.0,0.79,-21.19,50.48,18.60,149.0,-0.53,0.85,25.07,OX40N,NTT,CERAMICS
41,INDIGOPNTS,1408.0,108.15,40.0,M-SC,2.18,136561.0,-37998.0,38032.0,0.36,-21.77,27.85,0.02,221.0,-1.00,1.03,18.78,OX40N,NTT,PAINTS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TCS,4998.00,-26.76,49.0,H-LC,15.58,291344.0,-54612.0,178477.0,0.18,-15.79,61.26,35.80,1.0,-0.31,2.20,3.21,X40,BTT,IT
43,INFY,2275.00,-19.24,45.0,H-LC,11.73,314644.0,1756.0,169939.0,-1.53,0.56,54.01,54.87,3.0,0.01,2.37,7.25,X40,BTT,IT
45,ITC,452.00,-37.92,50.0,H-LC,2.99,199977.0,-161.0,21058.0,2.01,-0.08,10.53,10.44,4.0,-0.01,1.51,5.48,X40,NTT,FMCG
87,VBL,671.64,-13.42,42.0,H-LC,8.07,311950.0,-3892.0,116545.0,-1.76,-1.23,37.36,35.67,5.0,-0.03,2.35,12.45,X40N,ATH,FMCG
1,ABB,7934.00,-42.20,35.0,H-LC,13.46,241752.0,-19867.0,139080.0,0.72,-7.59,57.53,45.57,7.0,-0.14,1.82,2.87,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.0,13.45,61.0,H-MC,15.07,161936.0,35736.0,10445.0,-0.28,28.32,6.45,36.59,108.0,3.42,1.22,44.43,XY24,NTT,HEALTHCARE
27,DMART,5201.0,-6.86,74.0,H-LC,12.85,232505.0,48024.0,22344.0,0.96,26.03,9.61,38.14,38.0,2.15,1.75,39.40,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,13.45,61.0,H-MC,15.07,161936.0,35736.0,10445.0,-0.28,28.32,6.45,36.59,108.0,3.42,1.22,44.43,XY24,NTT,HEALTHCARE
14,BLUESTARCO,2326.38,9.48,59.0,H-MC,4.04,189000.0,24330.0,43640.0,0.53,14.78,23.09,41.28,89.0,0.56,1.43,23.94,X40N,ATH,AC
88,VOLTAS,1918.49,-1.64,60.0,H-MC,6.33,206400.0,14658.0,81384.0,1.23,7.64,39.43,50.08,99.0,0.18,1.56,14.96,XY25,ATH,AC
34,HEROMOTOCO,5949.07,-3.47,71.0,H-MC,15.85,178689.0,27776.0,29537.0,0.27,18.41,16.53,37.97,93.0,0.94,1.35,48.37,AR,ATH,AUTO
24,DABUR,735.00,-6.72,53.0,H-MC,3.77,202898.0,5870.0,83756.0,1.25,2.98,41.28,45.49,102.0,0.07,1.53,16.41,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,-3.47,71.0,H-MC,15.85,178689.0,27776.0,29537.0,0.27,18.41,16.53,37.97,93.0,0.94,1.35,48.37,AR,ATH,AUTO
40,INDIAMART,4850.92,-51.09,49.0,H-SC,12.47,136745.0,13409.0,120349.0,0.09,10.87,88.01,108.45,119.0,0.11,1.03,35.66,AR,ATH,MISC
89,WHIRLPOOL,2270.00,-41.67,51.0,M-SC,5.59,98558.0,7060.0,71691.0,1.35,7.72,72.74,86.07,223.0,0.10,0.74,43.12,XR,NTT,DURABLES
90,WIPRO,420.00,-11.98,51.0,M-LC,7.72,154876.0,3931.0,105517.0,-0.34,2.60,68.13,72.51,53.0,0.04,1.17,8.78,XR,NTT,IT
59,RAJOOENG,143.33,-32.99,38.0,H-SC,10.00,101530.0,-970.0,41800.0,0.14,-0.95,41.17,39.83,114.0,-0.02,0.77,0.52,AR,ATH,MISC


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ALKYLAMINE,4546.37,-6.86,36.0,H-SC,16.51,86744.0,-14219.0,108751.0,-0.32,-14.08,125.37,93.63,148.0,-0.13,0.65,29.20,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-39.64,71.0,L-SC,26.76,89472.0,-24077.0,64125.0,9.54,-21.20,71.67,35.27,268.0,-0.38,0.67,118.98,XR,NTT,HOTELS
52,MASFIN,398.61,-18.70,47.0,H-SC,12.72,92175.0,-5805.0,27404.0,0.23,-5.92,29.73,22.05,152.0,-0.21,0.70,33.98,XR,ATH,FINANCE
35,HINDALCO,756.01,-18.25,56.0,H-LC,7.63,112640.0,8328.0,8324.0,0.37,7.98,7.39,15.96,9.0,1.00,0.85,26.23,X5K,ATH,METALS
82,UJJIVANSFB,60.00,82.03,41.0,H-SC,14.60,115182.0,-27297.0,46821.0,0.54,-19.16,40.65,13.70,163.0,-0.58,0.87,35.95,OX40N,NTT,BANKS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-39.64,71.0,L-SC,26.76,89472.0,-24077.0,64125.0,9.54,-21.20,71.67,35.27,268.0,-0.38,0.67,118.98,XR,NTT,HOTELS
27,DMART,5201.00,-6.86,74.0,H-LC,12.85,232505.0,48024.0,22344.0,0.96,26.03,9.61,38.14,38.0,2.15,1.75,39.40,X40N,NTT,FMCG
34,HEROMOTOCO,5949.07,-3.47,71.0,H-MC,15.85,178689.0,27776.0,29537.0,0.27,18.41,16.53,37.97,93.0,0.94,1.35,48.37,AR,ATH,AUTO
14,BLUESTARCO,2326.38,9.48,59.0,H-MC,4.04,189000.0,24330.0,43640.0,0.53,14.78,23.09,41.28,89.0,0.56,1.43,23.94,X40N,ATH,AC
23,COLPAL,3767.14,0.29,58.0,H-MC,10.60,182528.0,-29213.0,111306.0,3.42,-13.80,60.98,38.77,84.0,-0.26,1.38,8.64,XY25,ATH,FMCG


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.64
1,25,43.09
2,50,72.67


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    37.02
LC    36.75
MC    26.18
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.13
X40      15.51
X40N     14.24
XY25      9.79
AR        9.32
XR        8.95
OX40N     7.31
X5K       2.23
X200      1.74
MH        1.66
SR        1.07
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.63
H-MC    23.32
H-SC    22.28
M-SC    13.17
M-LC     7.07
M-MC     2.54
L-SC     1.57
L-LC     1.05
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.19,-1.39,34.42
IT,13.35,-16.60,74.43
BANKS,7.31,-17.54,67.45
FINANCE,7.23,-25.79,72.17
PAINTS,5.95,-17.01,43.20
MISC,5.83,-18.72,86.77
ELECTRICAL,5.65,-9.68,70.13
AUTO,4.55,-12.22,61.05
AC,3.72,1.71,31.58


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2904850.0,23
AR,1244640.0,10
XR,1105246.0,13
X40,911957.0,10
X40N,542573.0,11
OX40N,487899.0,10
XY25,443869.0,7
SR,200626.0,2
X5K,142362.0,2


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2818849.0,27
M-SC,1498444.0,17
H-LC,1432240.0,17
H-MC,1285660.0,18
M-LC,440484.0,5
M-MC,340660.0,2
L-SC,251732.0,3
L-MC,60339.0,1
L-LC,44866.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,949852.0,6
M-SC,XY24,881321.0,7
H-SC,AR,670745.0,5
H-LC,X40,598518.0,5
H-SC,XR,534315.0,6
H-MC,XY24,496391.0,5
H-LC,AR,383275.0,2
M-MC,XY24,340660.0,2
H-MC,X40,231143.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 56.0 seconds
